In [1]:
# Importing packages
import numpy as np
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import *
from pyspark.sql import functions as F

spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.157:7077") \
        .appName("Group14_DE")\
        .config("spark.executor.instances", "4")\
        .config("spark.executor.memory", "12g")\
        .config("spark.executor.cores", "4")\
        .config("spark.driver.memory", "2g")\
        .config("spark.memory.fraction", "0.8")\
        .config("spark.memory.storageFraction", "0.5")\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.minExecutors", "1")\
        .config("spark.dynamicAllocation.maxExecutors", "4")\
        .config("spark.dynamicAllocation.initialExecutors", "2")\
        .config("spark.dynamicAllocation.executorIdleTimeout", "60s")\
        .config("spark.sql.shuffle.partitions", "32")\
        .getOrCreate()
# Retreiving the SparkContext
sc = spark_session.sparkContext

# Adjusting the log messages 
sc.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/14 13:07:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Load the dataset
df = spark_session.read.json("hdfs://192.168.2.157:9000/user/ubuntu/input/corpus-webis-tldr-17.json").persist()

In [4]:
# Print the schema to understand the structure of the dataset
df.printSchema()

root
 |-- author: string (nullable = true)
 |-- body: string (nullable = true)
 |-- content: string (nullable = true)
 |-- content_len: long (nullable = true)
 |-- id: string (nullable = true)
 |-- normalizedBody: string (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- subreddit_id: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- summary_len: long (nullable = true)
 |-- title: string (nullable = true)



In [5]:
from pyspark.sql.functions import col

# get all the subreddits in a df
subreddits_df = df.select(col("subreddit")).groupBy("subreddit").count().orderBy(col("count").desc())

subreddits_df.show(10)

24/03/14 13:34:05 ERROR TaskSchedulerImpl: Lost executor 0 on 192.168.2.157: worker lost: Not receiving heartbeat for 60 seconds
24/03/14 13:35:19 ERROR TaskSchedulerImpl: Lost executor 13 on 192.168.2.157: Command exited with code 137
24/03/14 13:37:05 ERROR TaskSchedulerImpl: Lost executor 14 on 192.168.2.157: Command exited with code 137


+-------------------+------+
|          subreddit| count|
+-------------------+------+
|          AskReddit|589947|
|      relationships|352049|
|    leagueoflegends|109307|
|               tifu| 52219|
|relationship_advice| 50416|
|              trees| 47286|
|             gaming| 43851|
|            atheism| 43268|
|      AdviceAnimals| 40783|
|              funny| 40171|
+-------------------+------+
only showing top 10 rows



In [6]:
from pyspark.sql.functions import length

# Map each record to a DataFrame containing the length of the summary and the length of the text
lengths_df = df.select(length("summary").alias("summary_length"), length("content").alias("content_length"))

# Calculate the correlation between the two columns
correlation = lengths_df.stat.corr("summary_length", "content_length")

print("Correlation between summary length and text length:", correlation)

# Stop the SparkSession
spark_session.stop()

24/03/14 13:38:48 ERROR TaskSchedulerImpl: Lost executor 15 on 192.168.2.157: Command exited with code 137


Correlation between summary length and text length: 0.32852918253101326
